## 基本圖片判斷+匯出圖片結果

In [4]:
import os
from ultralytics import YOLO
from PIL import Image, ImageDraw, ImageFont

model = YOLO("./yolov8n.pt")

furniture_classes = ["chair", "couch", "bed", "dining table", "tv", "microwave", "oven", "sink", "refrigerator"]

root_folder_path = "C:\\Users\\user\\OneDrive\\桌面\\爬蟲\\gold_house"
output_dir = "C:\\Users\\user\\OneDrive\\桌面\\爬蟲\\output_results"

processed_count = 0
max_process = 3

for subdir in os.listdir(root_folder_path):
    if processed_count >= max_process:
        break
    
    subdir_path = os.path.join(root_folder_path, subdir)
    
    if os.path.isdir(subdir_path):
        output_subdir = os.path.join(output_dir, subdir)
        os.makedirs(output_subdir, exist_ok=True)
        
        for filename in os.listdir(subdir_path):
            if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".jpeg"):
                img_path = os.path.join(subdir_path, filename)

                results = model(img_path)

                print(f"Results for {filename} in {subdir}:")

                img = Image.open(img_path)
                draw = ImageDraw.Draw(img)
                font = ImageFont.load_default()

                for result in results:
                    classes = result.boxes.cls.cpu().numpy()
                    coordinates = result.boxes.xyxy.cpu().numpy()
                    confidences = result.boxes.conf.cpu().numpy()  # 信心度

                    for cls, (x1, y1, x2, y2), confidence in zip(classes, coordinates, confidences):
                        class_name = model.names[int(cls)]  
                        
                        if class_name in furniture_classes:
                            draw.rectangle([x1, y1, x2, y2], outline="red", width=2)
                            text = f"Class: {class_name}, Conf: {confidence:.2f}"
                            draw.text((x1, y1 - 10), text, fill="red", font=font)
                            
                            print(f"Class: {class_name}, Coordinates: ({x1}, {y1}, {x2}, {y2}), Confidence: {confidence:.2f}")

                output_path = os.path.join(output_subdir, filename)
                img.save(output_path)

                print()

        processed_count += 1



image 1/1 C:\Users\user\OneDrive\\\gold_house\10305143\image1.jpg: 480x640 1 chair, 2 dining tables, 1 tv, 1 remote, 1 cell phone, 180.3ms
Speed: 8.1ms preprocess, 180.3ms inference, 5.0ms postprocess per image at shape (1, 3, 480, 640)
Results for image1.jpg in 10305143:
Class: chair, Coordinates: (696.0425415039062, 346.7735290527344, 846.7531127929688, 481.3448181152344), Confidence: 0.71
Class: dining table, Coordinates: (534.5136108398438, 428.48016357421875, 836.0650024414062, 553.759521484375), Confidence: 0.41
Class: tv, Coordinates: (0.0, 298.88116455078125, 126.8696060180664, 572.13720703125), Confidence: 0.40
Class: dining table, Coordinates: (534.79052734375, 427.9256896972656, 832.4703369140625, 749.3265380859375), Confidence: 0.31


image 1/1 C:\Users\user\OneDrive\\\gold_house\10305143\image2.jpg: 640x608 1 refrigerator, 226.6ms
Speed: 5.0ms preprocess, 226.6ms inference, 5.0ms postprocess per image at shape (1, 3, 640, 608)
Results for image2.jpg in 10305143:
Class: re

## 圖片相似度比對

In [17]:
import os
from ultralytics import YOLO
import concurrent.futures

# 初始化 YOLO 模型
model = YOLO('yolov8n.pt')

def detect_objects(image_path):
    try:
        results = model(image_path)
        
        features = []
        for result in results:
            for box in result.boxes:
                xmin, ymin, xmax, ymax = box.xyxy[0].tolist()
                confidence = box.conf[0].item()
                class_id = box.cls[0].item()
                
                features.append({
                    'bbox': [xmin, ymin, xmax, ymax],
                    'confidence': confidence,
                    'class_id': int(class_id)
                })
        return features
    except Exception as e:
        print(f"Error detecting objects in image {image_path}: {e}")
        return None

def calculate_iou(box1, box2):
    x1, y1, x2, y2 = box1
    x1_, y1_, x2_, y2_ = box2
    
    xi1 = max(x1, x1_)
    yi1 = max(y1, y1_)
    xi2 = min(x2, x2_)
    yi2 = min(y2, y2_)
    
    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    
    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (x2_ - x1_) * (y2_ - y1_)
    
    union_area = box1_area + box2_area - inter_area
    
    iou = inter_area / union_area
    return iou

def calculate_image_similarity_yolo(features1, features2, iou_threshold=0.5):
    for obj1 in features1:
        for obj2 in features2:
            iou = calculate_iou(obj1['bbox'], obj2['bbox'])
            if iou > iou_threshold and obj1['class_id'] == obj2['class_id']:
                return True
    
    return False

def compare_yolo_folders(folder1, folder2, similarity_threshold=0.5):
    images1 = [os.path.join(folder1, file) for file in os.listdir(folder1) if file.endswith(('.jpg', '.jpeg', '.png'))]
    images2 = [os.path.join(folder2, file) for file in os.listdir(folder2) if file.endswith(('.jpg', '.jpeg', '.png'))]

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for image1 in images1:
            for image2 in images2:
                futures.append(executor.submit(compare_images_yolo, image1, image2, similarity_threshold))

        for future in concurrent.futures.as_completed(futures):
            if future.result():
                return True

    return False


folders_to_process = [
    "C:\\Users\\user\\OneDrive\\桌面\\爬蟲\\gold_house\\8713071",
    "C:\\Users\\user\\OneDrive\\桌面\\爬蟲\\gold_house\\8713071 - 複製",
    "C:\\Users\\user\\OneDrive\\桌面\\爬蟲\\gold_house\\8957187"
]

similar = compare_yolo_folders(folders_to_process[0], folders_to_process[1])
if similar:
    print("8713071 and 8713071 - 複製 are similar.")
else:
    print("8713071 and 8713071 - 複製 are not similar.")

similar = compare_yolo_folders(folders_to_process[0], folders_to_process[2])
if similar:
    print("8713071 and 8957187 are similar.")
else:
    print("8713071 and 8957187 are not similar.")

similar = compare_yolo_folders(folders_to_process[1], folders_to_process[2])
if similar:
    print("8713071 - 複製 and 8957187 are similar.")
else:
    print("8713071 - 複製 and 8957187 are not similar.")













Ultralytics YOLOv8.2.42  Python-3.11.7 torch-2.3.0+cpu CPU (Intel Core(TM) i5-9300H 2.40GHz)
Ultralytics YOLOv8.2.42  Python-3.11.7 torch-2.3.0+cpu CPU (Intel Core(TM) i5-9300H 2.40GHz)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs
image 1/1 C:\Users\user\OneDrive\\\gold_house\8713071\image1.jpg: 480x640 1 couch, 1 tv, 1 refrigerator, 491.1ms
Speed: 5.9ms preprocess, 491.1ms inference, 10.0ms postprocess per image at shape (1, 3, 480, 640)


Ultralytics YOLOv8.2.42  Python-3.11.7 torch-2.3.0+cpu CPU (Intel Core(TM) i5-9300H 2.40GHz)
Ultralytics YOLOv8.2.42  Python-3.11.7 torch-2.3.0+cpu CPU (Intel Core(TM) i5-9300H 2.40GHz)
Ultralytics YOLOv8.2.42  Python-3.11.7 torch-2.3.0+cpu CPU (Intel Core(TM) i5-9300H 2.40GHz)
Ultralytics YOLOv8.2.42  Python-3.11.7 torch-2.3.0+cpu CPU (Intel Core(TM) i5-9300H 2.40GHz)
Ultralytics YOLOv8.2.42  Python-3.11.7 torch-2.3.0+cpu 

In [ ]:
import os
from itertools import combinations
from PIL import Image
import numpy as np
from skimage.metrics import structural_similarity as ssim
import concurrent.futures

folders_to_process = [
    "C:\\Users\\user\\OneDrive\\桌面\\爬蟲\\gold_house\\8713071",
    "C:\\Users\\user\\OneDrive\\桌面\\爬蟲\\gold_house\\8713071 - 複製",
    "C:\\Users\\user\\OneDrive\\桌面\\爬蟲\\gold_house\\8957187"
]

# 直接使用yolo來做，0.5左右才有我想要的結果
image_similarity_threshold = 0.5
folder_similarity_threshold = 0.5

def preprocess_image(image_path, target_size=(1000, 1000)):
    try:
        image = Image.open(image_path).convert('L')
        image = image.resize(target_size) 
        image = np.array(image) / 255.0   
        return image
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return None

# 利用SSIM結構相似性比較:影像品質的衡量上更能符合人眼對影像品質的判斷。
def calculate_image_similarity(image_path1, image_path2):
    image1 = preprocess_image(image_path1)
    image2 = preprocess_image(image_path2)
    
    if image1 is None or image2 is None:
        return 0.0
    
    similarity_score = ssim(image1, image2, data_range=image1.max() - image1.min())
    
    return similarity_score

# 比較資料夾內圖像
def compare_folders(folder1, folder2):
    images1 = [os.path.join(folder1, file) for file in os.listdir(folder1) if file.endswith(('.jpg', '.jpeg', '.png'))]
    images2 = [os.path.join(folder2, file) for file in os.listdir(folder2) if file.endswith(('.jpg', '.jpeg', '.png'))]
    
    total_pairs = 0
    similarity_sum = 0.0
    
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for image1 in images1:
            for image2 in images2:
                futures.append(executor.submit(calculate_image_similarity, image1, image2))
        
        for future in concurrent.futures.as_completed(futures):
            similarity_sum += future.result()
            total_pairs += 1
    
    if total_pairs > 0:
        average_similarity = similarity_sum / total_pairs
    else:
        average_similarity = 0.0
    
    return average_similarity

similar_folders = set()

# 比對
for folder1, folder2 in combinations(folders_to_process, 2):
    folder_similarity = compare_folders(folder1, folder2)
    
    if folder_similarity >= folder_similarity_threshold:
        similar_folders.add(os.path.basename(folder1))
        similar_folders.add(os.path.basename(folder2))

if similar_folders:
    print("Similar folders found:")
    for folder in similar_folders:
        print(folder)
else:
    print("No similar folders found.")
